In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import os

train_data_path = "~/aiffel/dktc/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [2]:
train_data = train_data.sample(frac = 1, random_state = 42)

def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()

    return sentence

def check_class(it):
    if '협박' in it:
        return 0
    elif '갈취' in it:
        return 1
    elif '직장 내 괴롭힘' in it:
        return 2
    elif '기타 괴롭힘' in it:
        return 3
    else:
        return 4

train_data['data'] = train_data['conversation'].apply(lambda it : preprocess_sentence(it))
train_data['label'] = train_data['class'].apply(lambda it: check_class(it))

train_data = train_data[['data', 'label']]

train_data.head()

,data,label
3136,야 . 절뚝이 왜 ? 똑바로 걸어다녀라 어려서 소아마비 앓아서 그래 절뚝절뚝 방아깨...,3
2118,씨 이번에 또 틀렸잖아 . 죄송합니다 . 다시 확인하겠습니다 . 동기인 씨는 한번 ...,2
1811,야 왜 쳐 ? 나 가만히 있었는데 아니 왜 치냐고 내가 거짓말하는거야 ? 왜 또 이...,3
70,니가 드디어 아들을 낳으니 내가 속이 다 시원하다 . 네 아버지 . 내가 그동안 명...,3
2609,야 옆반에 걔 봤냐 ? 어제 전학 온애 ? ? 어 걸어 다니면서 맨날 얼굴 툭툭 튀...,3


In [19]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_93/3596193378.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [4]:
# Set augmentation parameters
alpha_sr = 0.1
alpha_ri = 0.1
alpha_rs = 0.1
p_rd = 0.1
num_aug = 4


# Apply text data augmentation
def apply_augmentation(data, labels, num_aug):
    augmented_data = []
    augmented_labels = []
    for sentence, label in zip(train_data, labels):
        augmented_sentences = EDA(sentence, alpha_sr, alpha_ri, alpha_rs, p_rd, num_aug)
        augmented_labels.extend([label] * len(augmented_sentences))
        augmented_data.extend(augmented_sentences)
    return augmented_data, augmented_labels


In [5]:
augmented_sentences, augmented_labels = apply_augmentation(train_data['data'], train_data['label'], num_aug)

In [6]:
# 합쳐진 데이터 및 라벨 생성
combined_data = train_data['data'].tolist() + augmented_sentences
combined_labels = train_data['label'].tolist() + augmented_labels

# 합쳐진 데이터와 라벨을 DataFrame으로 변환
combined_df = pd.DataFrame({'data': combined_data, 'label': combined_labels})

print(combined_df.head())  # 합쳐진 데이터셋 확인


                                                data  label
0  야 . 절뚝이 왜 ? 똑바로 걸어다녀라 어려서 소아마비 앓아서 그래 절뚝절뚝 방아깨...      3
1  씨 이번에 또 틀렸잖아 . 죄송합니다 . 다시 확인하겠습니다 . 동기인 씨는 한번 ...      2
2  야 왜 쳐 ? 나 가만히 있었는데 아니 왜 치냐고 내가 거짓말하는거야 ? 왜 또 이...      3
3  니가 드디어 아들을 낳으니 내가 속이 다 시원하다 . 네 아버지 . 내가 그동안 명...      3
4  야 옆반에 걔 봤냐 ? 어제 전학 온애 ? ? 어 걸어 다니면서 맨날 얼굴 툭툭 튀...      3


In [7]:
# Print a few examples of combined data text
for idx, text in enumerate(combined_data[:5]):  
    print(f"Example {idx + 1}: {text}")

Example 1: 야 . 절뚝이 왜 ? 똑바로 걸어다녀라 어려서 소아마비 앓아서 그래 절뚝절뚝 방아깨비냐 ? 뚱뚱한 방아깨비네 사람이 아닌 곤충이었어 놀리지마 곤충이 말을 하네 제발 그만해
Example 2: 씨 이번에 또 틀렸잖아 . 죄송합니다 . 다시 확인하겠습니다 . 동기인 씨는 한번 알려주면 끝인데 자네는 매번 알려줘도 이래 . 죄송하지만 팀장님한테 확인받고 한 작업입니다 . 뭐 그럼 내가 일부러 그런다는 거야 . 그건 아니지만 . 그런 식으로 일할거면 회사 관둬요 . 누가 보면 내가 괴롭힌다고 생각하겠어 그런 말씀은 지나치십니다 . 그럼 알아서 잘 하던가 . 눈치도 없이 . 다음부터는 다시 확인하겠습니다 . 팀장님도 말씀 가려서 해주세요 .
Example 3: 야 왜 쳐 ? 나 가만히 있었는데 아니 왜 치냐고 내가 거짓말하는거야 ? 왜 또 이래 ? 야 얘가 왜 또 이러냐는데 이제 그만해 그만하래 웃기다 너 하루 이틀도 아니고 맨날 이런식으로 몰아가잖아 그냥 재밌어서 그래 장난이야 너한텐 장난이겠지
Example 4: 니가 드디어 아들을 낳으니 내가 속이 다 시원하다 . 네 아버지 . 내가 그동안 명절마다 사돈에 안부 전화할때마다 네가 네 역할을 다하지 못해서 얼마나 창피한 줄 알았니 ? 얼굴도 들지도 못했다 . 아 아버지 . 하아 . 제가 더 만 고생이 심했어요 . 니가 드디어 아기를 가졌으니 다행이야 . 마침 아들이니 더욱 다행이고 . 네 아버지 . 진짜 이 와서 너무 행복해요 . 근데 너 시험관하느라 몇 년 허비 한 거 알지 ? 니 나이가 몇인데 빨랑 둘째 가져 . 아버지 . 저 진짜 힘들게 아기 가진거고 둘째 생각 없어요 . 네가 큰 딸인데 그게 무슨 소리야 ? 빨랑 둘째 가져라 . 애 키우느라 쉰다매 . 어차피 시간도 많은데 병원 다니면서 힘써라 . 아버지 . 애 키우는 것도 힘들어요 . 정말 .
Example 5: 야 옆반에 걔 봤냐 ? 어제 전학 온애 ? ? 어 걸어 다니면서 맨날 얼굴 툭툭 튀는것처럼 흔들던데 ? 그니까 그거 겁나 웃기지

In [8]:
# Split the combined data into training and testing sets
X_train_combined, X_test_combined, Y_train_combined, Y_test_combined = train_test_split(
    combined_data,
    combined_labels,
    test_size=0.2,
    random_state=42,
    stratify=combined_labels
)

In [9]:
tfidf = TfidfVectorizer(min_df = 5, ngram_range = (1,3))
X_train_tf = tfidf.fit_transform(X_train_combined)

svc_model = LinearSVC(random_state = 0, tol = 1e-5)
svc_model.fit(X_train_tf, Y_train_combined)


X_test_tf = tfidf.transform(X_test_combined)

svc_pred = svc_model.predict(X_test_tf)


svc_accuracy = accuracy_score(Y_test_combined, svc_pred)

print("Linear Support Vector Machine Accuracy:", svc_accuracy)


Linear Support Vector Machine Accuracy: 0.8244949494949495


1) rs(random swap), rd(random deletion)

In [10]:
def EDA(sentence, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1


	n_rs = max(1, int(alpha_rs*num_words))


	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_93/1606832301.py:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [11]:
# Set augmentation parameters
alpha_sr = 0.1
p_rd = 0.1
num_aug = 4


# Apply text data augmentation
def apply_augmentation(data, labels, num_aug):
    augmented_data = []
    augmented_labels = []
    for sentence, label in zip(train_data, labels):
        augmented_sentences = EDA(sentence, alpha_rs, p_rd, num_aug)
        augmented_labels.extend([label] * len(augmented_sentences))
        augmented_data.extend(augmented_sentences)
    return augmented_data, augmented_labels


In [12]:
augmented_sentences, augmented_labels = apply_augmentation(train_data['data'], train_data['label'], num_aug)

In [13]:
# 합쳐진 데이터 및 라벨 생성
combined_data = train_data['data'].tolist() + augmented_sentences
combined_labels = train_data['label'].tolist() + augmented_labels

# 합쳐진 데이터와 라벨을 DataFrame으로 변환
combined_df = pd.DataFrame({'data': combined_data, 'label': combined_labels})

print(combined_df.head())  # 합쳐진 데이터셋 확인


                                                data  label
0  야 . 절뚝이 왜 ? 똑바로 걸어다녀라 어려서 소아마비 앓아서 그래 절뚝절뚝 방아깨...      3
1  씨 이번에 또 틀렸잖아 . 죄송합니다 . 다시 확인하겠습니다 . 동기인 씨는 한번 ...      2
2  야 왜 쳐 ? 나 가만히 있었는데 아니 왜 치냐고 내가 거짓말하는거야 ? 왜 또 이...      3
3  니가 드디어 아들을 낳으니 내가 속이 다 시원하다 . 네 아버지 . 내가 그동안 명...      3
4  야 옆반에 걔 봤냐 ? 어제 전학 온애 ? ? 어 걸어 다니면서 맨날 얼굴 툭툭 튀...      3


In [14]:
# Split the combined data into training and testing sets
X_train_combined, X_test_combined, Y_train_combined, Y_test_combined = train_test_split(
    combined_data,
    combined_labels,
    test_size=0.2,
    random_state=42,
    stratify=combined_labels
)

In [15]:
tfidf = TfidfVectorizer(min_df = 5, ngram_range = (1,3))
X_train_tf = tfidf.fit_transform(X_train_combined)

svc_model = LinearSVC(random_state = 0, tol = 1e-5)
svc_model.fit(X_train_tf, Y_train_combined)


X_test_tf = tfidf.transform(X_test_combined)

svc_pred = svc_model.predict(X_test_tf)


svc_accuracy = accuracy_score(Y_test_combined, svc_pred)

print("Linear Support Vector Machine Accuracy:", svc_accuracy)


Linear Support Vector Machine Accuracy: 0.8244949494949495


2) sr(synonym_replacement)

In [16]:
def EDA(sentence, alpha_sr=0.1, num_aug=5):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))


<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_93/2920791182.py:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [20]:
# Set augmentation parameters
alpha_sr = 0.1
num_aug = 4


# Apply text data augmentation
def apply_augmentation(data, labels, num_aug):
    augmented_data = []
    augmented_labels = []
    for sentence, label in zip(train_data, labels):
        augmented_sentences = EDA(sentence, alpha_sr, num_aug)
        augmented_labels.extend([label] * len(augmented_sentences))
        augmented_data.extend(augmented_sentences)
    
    return augmented_data, augmented_labels


In [21]:
augmented_sentences, augmented_labels = apply_augmentation(train_data['data'], train_data['label'], num_aug)

In [22]:
# 합쳐진 데이터 및 라벨 생성
combined_data = train_data['data'].tolist() + augmented_sentences
combined_labels = train_data['label'].tolist() + augmented_labels

# 합쳐진 데이터와 라벨을 DataFrame으로 변환
combined_df = pd.DataFrame({'data': combined_data, 'label': combined_labels})

print(combined_df.head())  # 합쳐진 데이터셋 확인


                                                data  label
0  야 . 절뚝이 왜 ? 똑바로 걸어다녀라 어려서 소아마비 앓아서 그래 절뚝절뚝 방아깨...      3
1  씨 이번에 또 틀렸잖아 . 죄송합니다 . 다시 확인하겠습니다 . 동기인 씨는 한번 ...      2
2  야 왜 쳐 ? 나 가만히 있었는데 아니 왜 치냐고 내가 거짓말하는거야 ? 왜 또 이...      3
3  니가 드디어 아들을 낳으니 내가 속이 다 시원하다 . 네 아버지 . 내가 그동안 명...      3
4  야 옆반에 걔 봤냐 ? 어제 전학 온애 ? ? 어 걸어 다니면서 맨날 얼굴 툭툭 튀...      3


In [23]:
# Split the combined data into training and testing sets
X_train_combined, X_test_combined, Y_train_combined, Y_test_combined = train_test_split(
    combined_data,
    combined_labels,
    test_size=0.2,
    random_state=42,
    stratify=combined_labels
)

In [24]:
tfidf = TfidfVectorizer(min_df = 5, ngram_range = (1,3))
X_train_tf = tfidf.fit_transform(X_train_combined)

svc_model = LinearSVC(random_state = 0, tol = 1e-5)
svc_model.fit(X_train_tf, Y_train_combined)


X_test_tf = tfidf.transform(X_test_combined)

svc_pred = svc_model.predict(X_test_tf)


svc_accuracy = accuracy_score(Y_test_combined, svc_pred)

print("Linear Support Vector Machine Accuracy:", svc_accuracy)


Linear Support Vector Machine Accuracy: 0.827455919395466
